In [1]:
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv("./Eviction Filings thru July2021.csv")

In [2]:
print(df.info(null_counts=True))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7923 entries, 0 to 7922
Data columns (total 82 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        7923 non-null   int64  
 1   Join_Count        7923 non-null   int64  
 2   TARGET_FID        7923 non-null   int64  
 3   Loc_name          5016 non-null   object 
 4   Status            7923 non-null   object 
 5   Score             7923 non-null   float64
 6   Match_type        7923 non-null   object 
 7   Match_addr        5016 non-null   object 
 8   User_fld          0 non-null      float64
 9   Addr_type         5016 non-null   object 
 10  ARC_Street        5498 non-null   object 
 11  ARC_City          5498 non-null   object 
 12  ARC_State         5498 non-null   object 
 13  ARC_ZIP           5448 non-null   float64
 14  case_numbe        7923 non-null   object 
 15  court_divi        7923 non-null   object 
 16  file_date         7923 non-null   object 


In [3]:
owner_count = df['owner_name'].value_counts()
print(owner_count)
owner_dict = owner_count.to_dict()

# How many owners with corresponded evictions
instances_list = {}
for value in owner_dict.values():
    if value not in instances_list:
        instances_list[value] = 1
    else:
        instances_list[value] += 1
print(instances_list)

WORCESTER HOUSING AUTHORITY      76
JEFFERSON AT EDGEWATER HILLS     44
LINCOLN STREET REALTY COMPANY    41
GPT-RG FALL RIVER LLC            28
WINDSOR GARDENS PROPCO LLC       27
                                 ..
AVILES,DAYANET                    1
FENWAY STUDIOS INC                1
DUNNING, BARBARA A TR             1
75 NORTH PAUL STREET LLC          1
PANIAGUA,CARMEN                   1
Name: owner_name, Length: 3116, dtype: int64
{76: 1, 44: 1, 41: 1, 28: 1, 27: 1, 24: 2, 22: 1, 21: 1, 20: 1, 18: 2, 17: 1, 15: 2, 14: 2, 13: 2, 12: 3, 11: 7, 10: 7, 9: 8, 8: 14, 7: 16, 6: 17, 5: 37, 4: 66, 3: 127, 2: 339, 1: 2456}


In [4]:
# evictions distribution
distribution = [0, 0, 0, 0, 0]
for key in instances_list.keys():
    if key >= 30:
        distribution[0] += instances_list[key]
    elif 20 <= key < 30:
        distribution[1] += instances_list[key]
    elif 10 <= key < 20:
        distribution[2] += instances_list[key]
    elif 2 <= key < 10:
        distribution[3] += instances_list[key]
    else:
        distribution[4] += instances_list[key]
print(distribution)

[3, 7, 26, 624, 2456]


3 evictings with more than 30 evictions; 7 evictings with 20-30 evictions; 26 evictings with 10-20 evictions; 624 evictions with 2-10 evictions; 2456 evictions with 1 eviction.

In [5]:
owner_with_much_evictions = []
for key in owner_dict.keys():
    if owner_dict[key] >= 20:
        owner_with_much_evictions.append(key)
print(owner_with_much_evictions)

['WORCESTER HOUSING AUTHORITY', 'JEFFERSON AT EDGEWATER HILLS', 'LINCOLN STREET REALTY COMPANY', 'GPT-RG FALL RIVER LLC', 'WINDSOR GARDENS PROPCO LLC', 'GS STONEGATE PROJECT OWNER LLC', 'FRAMINGHAM HOUSING AUTHORITY', '600 MAIN STREET,LLC', 'ARCHSTONE NORTH POINT I LLC,', 'CLARENDON HILL SOMERVILLE LP']


In [9]:
# dict with evictings & Corp
evictings_corp = df[['owner_name', 'Corp']]
evictings_corp = evictings_corp.drop_duplicates(subset=['owner_name'],keep='first',inplace=False)
evictings_corp_dict =  {key:values for key, values in zip(evictings_corp['owner_name'], evictings_corp['Corp'])}
is_corp = 0
not_corp = 0
for value in evictings_corp_dict.values():
    if value == True:
        is_corp += 1
    else:
        not_corp += 1
print(is_corp)
print(not_corp)
        

1292
1825


In [7]:
for item in owner_with_much_evictions:
    print((item, evictings_corp_dict[item]))

('WORCESTER HOUSING AUTHORITY', True)
('JEFFERSON AT EDGEWATER HILLS', False)
('LINCOLN STREET REALTY COMPANY', True)
('GPT-RG FALL RIVER LLC', True)
('WINDSOR GARDENS PROPCO LLC', True)
('GS STONEGATE PROJECT OWNER LLC', True)
('FRAMINGHAM HOUSING AUTHORITY', True)
('600 MAIN STREET,LLC', True)
('ARCHSTONE NORTH POINT I LLC,', True)
('CLARENDON HILL SOMERVILLE LP', True)
